In [ ]:
#| default_exp 12_dataset-statistics

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import scipy.sparse as sp

from sugar.core import *
from xcai.data import *

In [ ]:
data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/(mapped)LF-WikiSeeAlsoTitles-320K/'
save_dir = '/home/scai/phd/aiz218323/scratch/datasets/wikipedia/20250123/LF-WikiSeeAlsoTitles-320K/'

In [ ]:
def load_info(data_dir, type):
    ids, txt = load_raw_txt(f'{data_dir}/raw_data/{type}.raw.txt', encoding='latin-1')
    return {'identifier':ids, 'input_text':txt}
    

In [ ]:
def get_info(data_dir):
    trn_info = load_info(data_dir, 'train')
    tst_info = load_info(data_dir, 'test')
    
    return trn_info, tst_info
    

In [ ]:
def get_labels(data_dir):
    trn_mat = sp.load_npz(f'{data_dir}/trn_X_Y.npz')
    tst_mat = sp.load_npz(f'{data_dir}/tst_X_Y.npz')
    lbl_info = load_info(data_dir, 'label')
    
    return trn_mat, tst_mat, lbl_info
    

In [ ]:
def get_metadata(data_dir, meta_type, prefix=''):
    if len(prefix): prefix = f'{prefix}_'
    trn_mat = sp.load_npz(f'{data_dir}/{prefix}{meta_type}_trn_X_Y.npz')
    tst_mat = sp.load_npz(f'{data_dir}/{prefix}{meta_type}_tst_X_Y.npz')
    lbl_mat = sp.load_npz(f'{data_dir}/{prefix}{meta_type}_lbl_X_Y.npz')
    meta_info = load_info(data_dir, f'{prefix}{meta_type}')
    return trn_mat, tst_mat, lbl_mat, meta_info
    

In [ ]:
trn_info, tst_info = get_info(data_dir)
trn_mat, tst_mat, lbl_info = get_labels(save_dir)

In [ ]:
trn_dset = MainXCDataset(trn_info, trn_mat, lbl_info)
tst_dset = MainXCDataset(tst_info, tst_mat, lbl_info)

In [ ]:
trn_mat, tst_mat, lbl_mat, meta_info = get_metadata(save_dir, 'category')

In [ ]:
trn_meta_dset = MetaXCDataset('cat', trn_mat, lbl_mat, meta_info)
tst_meta_dset = MetaXCDataset('cat', tst_mat, lbl_mat, meta_info)

In [ ]:
lbl_mat

<312330x810093 sparse matrix of type '<class 'numpy.int64'>'
	with 1515714 stored elements in Compressed Sparse Row format>

In [ ]:
trn_dset = XCDataset(trn_dset, cat_meta=trn_meta_dset)
tst_dset = XCDataset(tst_dset, cat_meta=tst_meta_dset)

ValueError: `meta`(312330) and `data`(573503) should have the same number of labels.